# Westeros Tutorial - Introducing emissions (part1/2): Adding emission bounds

In the first part, the model chose not base the power system on wind power as electricity from  wind turbines was more expensive than electricity produced from coal. However, now we introduce emissions to investigate the impact of climate policy. Let's see what happens then.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
mp = ixmp.Platform()

2022-06-13 06:25:01,473  INFO at.ac.iiasa.ixmp.Platform:165 - Welcome to the IX modeling platform!
2022-06-13 06:25:01,477  INFO at.ac.iiasa.ixmp.Platform:166 -  connected to database 'jdbc:hsqldb:file:/Users/juliusbergmann/.local/share/ixmp/localdb/default' (user: ixmp)...


## Making a clone of the existing scenario 'baseline'

In [3]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "emission_bound",
    "introducing an upper bound on emissions",
    keep_solution=False,
)
scen.check_out()

In [4]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

## Introducing Emissions

In [5]:
# First we introduce the emission of CO2 and the emission category GHG
scen.add_set("emission", "CO2")
scen.add_cat("emission", "GHG", "CO2")

# Then we add new units to the model library (needed only once)
mp.add_unit("tCO2/kWa")
mp.add_unit("MtCO2")

# Last we add CO2 emissions to the coal powerplant
emission_factor = make_df(
    "emission_factor",
    node_loc=country,
    year_vtg=vintage_years,
    year_act=act_years,
    mode="standard",
    unit="tCO2/kWa",
    technology="coal_ppl",
    emission="CO2",
    value=7.4,
)
scen.add_par("emission_factor", emission_factor)

## Define a Bound on Emissions

The `"type_year": "cumulative"` assigns an upper bound on the *weighted average of emissions* over the entire time horizon.

In [6]:
#scen.remove_solution()
#scen.check_out()

In [7]:
scen.add_par(
    "bound_emission", [country, "GHG", "all", "cumulative"], value=100.0, unit="MtCO2"
)

## Time to Solve the Model

In [8]:
scen.commit(comment="Introducing emissions and setting an upper bound")
scen.set_as_default()

In [9]:
scen.solve()

--- Warning: The GAMS version [39.2.0] differs from the API version [24.8.3].
--- Job MESSAGE_run.gms Start 06/13/22 06:25:03 39.2.0 e0c5072f DEX-DEG x86 64bit/Mac OS X
--- Applying:
    /Library/Frameworks/GAMS.framework/Versions/39/Resources/gmsprmun.txt
--- GAMS Parameters defined
    Input /Users/juliusbergmann/opt/anaconda3/envs/Assignment3/lib/python3.9/site-packages/message_ix/model/MESSAGE_run.gms
    ScrDir /Users/juliusbergmann/opt/anaconda3/envs/Assignment3/lib/python3.9/site-packages/message_ix/model/225a/
    SysDir /Library/Frameworks/GAMS.framework/Versions/39/Resources/
    LogOption 4
    --in /Users/juliusbergmann/opt/anaconda3/envs/Assignment3/lib/python3.9/site-packages/message_ix/model/data/MsgData_Westeros_Electrified_emission_bound.gdx
    --out /Users/juliusbergmann/opt/anaconda3/envs/Assignment3/lib/python3.9/site-packages/message_ix/model/output/MsgOutput_Westeros_Electrified_emission_bound.gdx
    --iter /Users/juliusbergmann/opt/anaconda3/envs/Assignment3/li

ModelError: GAMS errored with return code 3:
    There was an execution error

For details, see the terminal output above, plus:
Listing   : /Users/juliusbergmann/opt/anaconda3/envs/Assignment3/lib/python3.9/site-packages/message_ix/model/MESSAGE_run.lst
Input data: /Users/juliusbergmann/opt/anaconda3/envs/Assignment3/lib/python3.9/site-packages/message_ix/model/data/MsgData_Westeros_Electrified_emission_bound.gdx

In [ ]:
scen.var("OBJ")["lvl"]

## Plotting Results

In [ ]:
from message_ix.reporting import Reporter
from message_ix.util.tutorial import prepare_plots

rep = Reporter.from_scenario(scen)
prepare_plots(rep)

### Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
rep.set_filters(t=["coal_ppl", "wind_ppl", "pv_ppl"])
rep.get("plot activity")

### Capacity

How much capacity of each plant is installed in each period?

In [ ]:
rep.get("plot capacity")

### Electricity Price

And how much does the electricity cost? These prices are in fact **shadow prices** taken from the **dual variables** of the model solution. They reflect the marginal cost of electricity generation (i.e., the additional cost of the system for supplying one more unit of electricity), which is in fact the marginal cost of the most expensive generator.  

Notice the drop in the price in the period of 710? 

The share of the activity of the coal power plant is in the period of 710 higher then the share in the period of 700. With lower electricity costs of the coal power plant, this leads to reduced electricity costs. To stay within the emission bounds with an increased electricity demand, in the upcoming period of 720, the capacity of the wind turbines increases drastically, while the capacity of the coal power plants remains the same. This results in the increased electricity price.

In [ ]:
rep.set_filters(t=None, c=["light"])
rep.get("plot prices")

## Close the connection to the database

In [ ]:
mp.close_db()